In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# temperature가 낮을수록 낮은 창의성을 갖는다.
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

In [35]:
# 기존 PromptTempalte

t = PromptTemplate.from_template("What is the capital of {country}")

t.format(country="France")

'What is the capital of France'

In [40]:
# FewShotPromptTemplate
# 모델에게 어떻게 대답해야 하는 지에 대한 예제를 AI 모델에게 주는 것이 
# prompt를 사용해서 어떻게 대답해야 하는지 알려주는 것보다 훨씬 좋음

# data는 db에서 가져올 수도 있음
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# chat.predict("What do you know abount France?")

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# LangChain이 알아서 각각의 예제 리스트들을 해당 prompt를 사용해 형식화
# suffix : 사용자의 질문이 어떻게 보여질 지 
# input_variables: template에서 사용할 입력변수가 무엇인지 지정. 유효성 검사를 받을 수 있음
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Germany?'

In [43]:
chain = prompt | chat

chain.invoke({"country":"Germany"})

AI: 
        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')